In [61]:
import pandas as pd
from itertools import permutations
import math
import os
import csv
import time

In [66]:
cargo_capacity_psv = 100
psv_speed = 10
max_platforms_in_one_voyage = 7

mappenavn = 'generated_datafiles_allroutes'

## Reads input data

In [63]:
demand = pd.read_csv('clustering/output_platforms_demand.csv', header=0, delimiter=';')
distances = pd.read_csv('clustering/output_distance_matrix_kmeans.csv', header=0, delimiter=';', index_col='from/to')

platforms_demand = dict(zip(demand['platform'], demand['avg_q'].replace(',', '.').astype(float)))
platforms_d = ['MON'] + demand['platform'].tolist() + ['MON']  # Add 'DUS' as start and end platforms

## Algorithm for generating the shortest routes

In [64]:
shortest_routes_dict = {}

def generate_routes(demand, distances):
    cargo_capacity = cargo_capacity_psv
    max_platforms = max_platforms_in_one_voyage + 2

    def dp(platform, cargo_remaining, route, visited):
        if cargo_remaining < 0:
            return
        if len(route) > max_platforms:
            return
        if platform == 'MON' and len(route) > 2:
            total_demand = sum(platforms_demand[p] for p in route[1:-1])
            if total_demand <= cargo_capacity:
                key = tuple(sorted(set(route)))
                total_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
                if key not in shortest_routes_dict or total_distance < shortest_routes_dict[key][1]:
                    duration_sailing = round((total_distance / psv_speed), 2)
                    duration_lossing = round(((total_demand * 1.389) / psv_speed), 2)
                    duration_sailing = round(duration_sailing, 2)
                    duration_lossing = round(duration_lossing, 2)
                    shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)
            return

        # Check if the current route is dominated
        current_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
        current_demand = sum(platforms_demand[p] for p in route[1:-1])
        
        # Check for dominance in existing routes
        for key, (existing_route, existing_distance, existing_demand, _, _) in shortest_routes_dict.items():
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return
        
        # Check for dominance in subsequent routes starting with the same platforms
        for existing_route in dominated_routes:
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return

        for next_platform in platforms_demand.keys():
            if next_platform != platform and next_platform not in visited:
                try:
                    distance_to_next = distances.loc[platform, next_platform]
                    new_cargo_remaining = cargo_remaining - platforms_demand[next_platform]
                    dp(next_platform, new_cargo_remaining, route + [next_platform], visited.union({next_platform}))
                except KeyError:
                    print("KeyError occurred for platform:", next_platform)
                    continue

    for r in range(3, min(len(platforms_demand) + 3, max_platforms + 3)):
        for route_combination in permutations(platforms_demand.keys(), r - 2):
            route = ['MON'] + list(route_combination) + ['MON']
            dp('MON', cargo_capacity, route, {'MON'})

            # Add the current route to the dominated routes set
            dominated_routes.add(tuple(route))

    return shortest_routes_dict

# Keep track of dominated routes to skip future routes starting with them
dominated_routes = set()
shortest_routes_dict = generate_routes(demand, distances)

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    print(f"Shortest Route: {route}, Total Distance: {round(distance,2)}, Total Demand: {demand}, Duration sailing: {duration_sailing}, Duration lossing: {duration_lossing}")

print(len(shortest_routes_dict))

Shortest Route: ['DUS', 'DAB', 'DUS'], Total Distance: 185.84, Total Demand: 40.0, Duration sailing: 18.58, Duration lossing: 5.56
Shortest Route: ['DUS', 'DRA', 'DUS'], Total Distance: 205.36, Total Demand: 14.0, Duration sailing: 20.54, Duration lossing: 1.94
Shortest Route: ['DUS', 'DSA', 'DUS'], Total Distance: 164.4, Total Demand: 29.0, Duration sailing: 16.44, Duration lossing: 4.03
Shortest Route: ['DUS', 'GKR', 'DUS'], Total Distance: 234.68, Total Demand: 18.0, Duration sailing: 23.47, Duration lossing: 2.5
Shortest Route: ['DUS', 'GUD', 'DUS'], Total Distance: 226.76, Total Demand: 18.0, Duration sailing: 22.68, Duration lossing: 2.5
Shortest Route: ['DUS', 'JSF', 'DUS'], Total Distance: 176.18, Total Demand: 40.0, Duration sailing: 17.62, Duration lossing: 5.56
Shortest Route: ['DUS', 'GRA', 'DUS'], Total Distance: 180.44, Total Demand: 14.0, Duration sailing: 18.04, Duration lossing: 1.94
Shortest Route: ['DUS', 'HDA', 'DUS'], Total Distance: 206.82, Total Demand: 18.0, Dur

## Saves the shortest routes left to csv files in the generated_datafiles folder. In total 858 routes

In [67]:
routes_only = [route for route, _, _, _, _ in shortest_routes_dict.values()]
distances_only = [distance for _, distance, _, _, _ in shortest_routes_dict.values()]
demand_only = [demand for _, _, demand, _, _ in shortest_routes_dict.values()]
duration_sailing = [duration_sailing for _, _, _, duration_sailing, _ in shortest_routes_dict.values()]
duration_lossing = [duration_lossing for _, _, _, _, duration_lossing in shortest_routes_dict.values()]

df_distances = pd.DataFrame({'Distance': distances_only})
df_demand = pd.DataFrame({'Demand': demand_only})
df_duration_sailing = pd.DataFrame({'Duration (hours)': duration_sailing})
df_duration_lossing = pd.DataFrame({'Duration (hours)': duration_lossing})

df_distances.to_csv(f'{mappenavn}/distances.csv', index=False)
df_demand.to_csv(f'{mappenavn}/demand.csv', index=False)
df_duration_sailing.to_csv(f'{mappenavn}/duration_sailing.csv', index=False)
df_duration_lossing.to_csv(f'{mappenavn}/duration_lossing.csv', index=False)
route_file = f'{mappenavn}/routes.csv'


def write_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)

write_to_csv(route_file, routes_only)

## Creating the reduced set of routes

In [ ]:
'''
import csv

def contains_required_routes(route, required_routes):
    return all(required_route in route for required_route in required_routes)

def main():
    hda_additional_routes1 = ["HDA", "SLB"]
    hda_additional_routes2 = ["HDA", "SLA"]
    hda_additional_routes3 = ["HDA", "DRA"]
    hda_additional_routes4 = ["HDA", "TEB"]
    
    dra_additional_routes1 = ["DRA", "DAB"]
    dra_additional_routes2 = ["DRA", "GRA"]

    gkr_additional_routes1 = ["GKR", "HDA"]

    teb_additional_routes1 = ["TEB", "DAB"]
    teb_additional_routes2 = ["TEB", "GRA"]

    sla_additional_routes1 = ["SLA", "DAB"]
    sla_additional_routes2 = ["SLA", "GRA"]

    slb_additional_routes1 = ["SLB", "DAB"]
    slb_additional_routes2 = ["SLB", "GRA"]

    routes_with_hda_slb = []
    routes_with_hda_sla = []
    routes_with_hda_dra = []
    routes_with_hda_teb = []

    routes_with_dra_dab = []
    routes_with_dra_gra = []
    routes_with_gkr_hda = []
    routes_with_teb_dab = []

    routes_with_teb_gra = []
    routes_with_sla_dab = []
    routes_with_sla_gra = []
    routes_with_slb_dab = []
    routes_with_slb_gra = []

    with open('generated_datafiles_allroutes/routes.csv', 'r') as file:
        csv_reader = csv.reader(file)
        for row_number, row in enumerate(csv_reader, start=1):
            if contains_required_routes(row, hda_additional_routes1):
                routes_with_hda_slb.append(row_number)
            if contains_required_routes(row, hda_additional_routes2):
                routes_with_hda_sla.append(row_number)
            if contains_required_routes(row, hda_additional_routes3):
                routes_with_hda_dra.append(row_number)
            if contains_required_routes(row, hda_additional_routes4):
                routes_with_hda_teb.append(row_number)
            if contains_required_routes(row, dra_additional_routes1):
                routes_with_dra_dab.append(row_number)
            if contains_required_routes(row, dra_additional_routes2):
                routes_with_dra_gra.append(row_number)

            if contains_required_routes(row, gkr_additional_routes1):
                routes_with_gkr_hda.append(row_number)

            if contains_required_routes(row, teb_additional_routes1):
                routes_with_teb_dab.append(row_number)
            if contains_required_routes(row, teb_additional_routes2):
                routes_with_teb_gra.append(row_number)
            if contains_required_routes(row, sla_additional_routes1):
                routes_with_sla_dab.append(row_number)
            if contains_required_routes(row, sla_additional_routes2):
                routes_with_sla_gra.append(row_number)
            if contains_required_routes(row, slb_additional_routes1):
                routes_with_slb_dab.append(row_number)
            if contains_required_routes(row, slb_additional_routes2):
                routes_with_slb_gra.append(row_number)
            
    # Finding rows where all combinations occur
    # Combine all lists into one
    all_routes = (
        routes_with_hda_slb + 
        routes_with_hda_sla +
        routes_with_hda_dra +
        routes_with_hda_teb +
        routes_with_dra_dab +
        routes_with_dra_gra +
        routes_with_gkr_hda +
        routes_with_teb_dab +
        routes_with_teb_gra +
        routes_with_sla_dab +
        routes_with_sla_gra +
        routes_with_slb_dab +
        routes_with_slb_gra
    )
    
    # Convert the combined list into a set to remove duplicates
    unique_routes = set(all_routes)

    # Convert the set back into a list if needed
    unique_routes_list = list(unique_routes)
    print((sorted(unique_routes_list)))
    print(len(unique_routes_list))

    def filter_routes(input_file, output_file, exclude_indices):
        with open(input_file, 'r', newline='') as f_in, \
            open(output_file, 'w', newline='') as f_out:
            reader = csv.reader(f_in)
            writer = csv.writer(f_out)

            for i, row in enumerate(reader):
                if i + 1 not in exclude_indices:  # Adjusting for 1-based indexing
                    writer.writerow(row)

    def filter_otherfiles(input_file, output_file, exclude_indices):
        with open(input_file, 'r', newline='') as f_in, \
            open(output_file, 'w', newline='') as f_out:
            reader = csv.reader(f_in)
            writer = csv.writer(f_out)

            for i, row in enumerate(reader):
                if i not in exclude_indices:  
                    writer.writerow(row)

    # Example usage
    input_file = 'generated_datafiles_allroutes/routes.csv'
    output_file = 'generated_datafiles_reducedroutes/routes.csv'

    input_file2 = 'generated_datafiles_allroutes/demand.csv'
    output_file2 = 'generated_datafiles_reducedroutes/demand.csv'

    input_file3 = 'generated_datafiles_allroutes/distances.csv'
    output_file3 = 'generated_datafiles_reducedroutes/distances.csv'

    input_file4 = 'generated_datafiles_allroutes/duration_lossing.csv'
    output_file4 = 'generated_datafiles_reducedroutes/duration_lossing.csv'

    input_file5 = 'generated_datafiles_allroutes/duration_sailing.csv'
    output_file5 = 'generated_datafiles_reducedroutes/duration_sailing.csv'

    filter_routes(input_file, output_file, unique_routes_list)
    filter_otherfiles(input_file2, output_file2, unique_routes_list)
    filter_otherfiles(input_file3, output_file3, unique_routes_list)
    filter_otherfiles(input_file4, output_file4, unique_routes_list)
    filter_otherfiles(input_file5, output_file5, unique_routes_list)

if __name__ == "__main__":
    main()
'''

'\nimport csv\n\ndef contains_required_routes(route, required_routes):\n    return all(required_route in route for required_route in required_routes)\n\ndef main():\n    hda_additional_routes1 = ["HDA", "SLB"]\n    hda_additional_routes2 = ["HDA", "SLA"]\n    hda_additional_routes3 = ["HDA", "DRA"]\n    hda_additional_routes4 = ["HDA", "TEB"]\n    \n    dra_additional_routes1 = ["DRA", "DAB"]\n    dra_additional_routes2 = ["DRA", "GRA"]\n\n    gkr_additional_routes1 = ["GKR", "HDA"]\n\n    teb_additional_routes1 = ["TEB", "DAB"]\n    teb_additional_routes2 = ["TEB", "GRA"]\n\n    sla_additional_routes1 = ["SLA", "DAB"]\n    sla_additional_routes2 = ["SLA", "GRA"]\n\n    slb_additional_routes1 = ["SLB", "DAB"]\n    slb_additional_routes2 = ["SLB", "GRA"]\n\n    routes_with_hda_slb = []\n    routes_with_hda_sla = []\n    routes_with_hda_dra = []\n    routes_with_hda_teb = []\n\n    routes_with_dra_dab = []\n    routes_with_dra_gra = []\n    routes_with_gkr_hda = []\n    routes_with_teb_d